In [1]:
%load_ext sql
%config SqlMagic.autopandas=False

/home/juliano/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


$Airport(\underline{apID}, name, city, country)$

$AirportDistance(\underline{apID1, apID2}, distance)$

$Airline(\underline{cID}, callSign, headquarters, mainHub, website)$

$Aircraft(\underline{acID}, type, avgSpeed, maxSpeed, range, serialNumber, firstFlightDate, manufacturer)$

$Employee(\underline{eID}, name, airline, type, rank, primary\_language, secondary\_language, startDate)$

$Flight(\underline{cID, flightNumber}, origin, destination, departureTime, arrivalTime)$

$FlightInstance(\underline{fID}, cID, flightNumber, date, acID, actualDepartureTime, actualArrivalTime)$

$CrewAssignment(\underline{fID, eID}, role)$

$PilotExpertise(\underline{eID}, aircraftType)$

$\Pi, \sigma, \bowtie, \bowtie_{condition}, \times, \cap, \cup, -, \rho, (such as \leq and \neq)$
and boolean operators $(\vee, \wedge and \neg).$

### 1. List for each airline the employees who have expertise in at least two different types of aircraft. Use the two-character airline code.


$EmpExpertise := (\Pi_{eID} Employee) \bowtie PilotExpertise$

$Experts(eID) := \Pi_{E1.eID}[(\rho_{E1} EmpExpertise) \bowtie_{E1.eID = E2.eID \wedge E1.aircraftType \neq E2.aircraftType} (\rho_{E2} EmpExpertise)]$

$Answer := Experts \bowtie Employee$

### 2. Find all the airports (airport code and name) that one can reach on the same day if one departs from Pearson airport (apID ’YYZ’) and is willing to take at most 3 flights. Assume that one needs at least 120 minutes between consecutive flights. 

**OBS**: If One.at + 120 <= FlInfo.dt, it means we can make both flights in the same day since every flight starts and finishes on the same day. Also, can I assume that an airport only travels to different places? maybe not


$FlInfo(cID, fnum, o, d, dt, at) := \Pi_{cID, flightNumber, origin, destination, departureTime, arrivalTime}Flight $

$One := \sigma_{o='YYZ'}FlInfo$

$Two := (One \bowtie_{((One.cID \neq FlInfo.cID)\; \vee \;(One.fnum \neq FlInfo.fnum))\; \wedge \;(One.d = FlInfo.o)\; \wedge \; (FlInfo.d \neq 'YYZ')} FlightInfo)$

$Two(cID, fnum, o, d, dt, at) := \Pi_{FlInfo.cID, FlInfo.fnum, FlInfo.o, FlInfo.d, FlInfo.dt, FlInfo.at}\sigma_{One.at + 120 \leq FlInfo.dt} Two$

$Three := (Two \bowtie_{((Two.cID \neq FlInfo.cID)\; \vee \;(Two.fnum \neq FlInfo.fnum))\wedge \;(Two.d = FlInfo.o)} FlightInfo)$

$Three(d) := \Pi_{FlInfo.d}\sigma_{Three.at + 120 \leq FlInfo.dt} Three$

$OneFlight(apID, name) := \Pi_{apID, name} (Airport \bowtie_{apID = d} (\Pi_{d} One))$

$TwoFlight(apID, name) := \Pi_{apID, name} (Airport \bowtie_{apID = d} (\Pi_{d} Two))$

$ThreeFlight(apID, name) := \Pi_{apID, name} (Airport \bowtie_{apID = d} (\Pi_{d} Three))$

$Answer(apID, name) := OneFlight\; \cup \; TwoFlight \; \cup \; ThreeFlight$


### 3. Find all flight instances on which no crew member speaks French (as either a first or second language). Return the airline, flight number, and the date of the flight instance. (Flight instances with no crew assigned satisfy the condition and should be returned as well.)

$AtLeastOneFrench(fID) :=  \Pi_{fID}(CrewAssignment \bowtie (\Pi_{eID}\sigma_{primary\_language='French' \; \vee \; secondary\_language='French'}Employee))$

$AllFlightsInstances (fID) := \Pi_{fID} FlightInstance$

$NoFrenchFlights(fID) := AllFlightsInstances - AtLeastOneFrench$

$Answer(cID, flightNumber, date) := \Pi_{cID, flightNumber, date} (FlightInstance \bowtie NoFrenchFlights)$

### 4. List all airports with more than 3 departing flights a day.

IDEA: Get all possible 4 tuples of flights, such that every flight is different from each other. Then, get the airport in which all agree, i.e. if all flights have the same 'origin', it means this airport has MORE than 3 departing flights (that is, at LEAST 4 flights departed a day, since a flight starts and ends at the same day).
The whole **Temp** relations are just to make sure every flight is paired only with different flights

$FlightSimplified(cid, flightNumber, origin) := \Pi_{cid, flightNumber, origin} Flight$

$QuadriCross := (\rho_{F1} FlightSimplified) \times (\rho_{F2} FlightSimplified) \times (\rho_{F3} FlightSimplified) \times (\rho_{F4} FlightSimplified)$

$Temp1a := \sigma_{\neg(F1.cid = F2.cid \wedge F1.flightNumber = F2.flightNumber) \wedge \neg(F1.cid = F3.cid \wedge F1.flightNumber = F3.flightNumber)}\;\; QuadriCross$

$Temp1b := \sigma_{\neg(F2.cid = F3.cid \wedge F2.flightNumber = F3.flightNumber)}\; Temp1a$

$Temp2 := \sigma_{\neg(F1.cid = F4.cid \wedge F1.flightNumber = F4.flightNumber) \wedge  \neg(F2.cid = F4.cid \wedge F2.flightNumber = F4.flightNumber)}\; \;Temp1b$

$AllDiffFlights := \sigma_{\neg(F3.cid = F4.cid \wedge F3.flightNumber = F4.flightNumber)}\; Temp2$


$Answer(airport) := \Pi_{F1.origin} \sigma_{F1.origin = F2.origin \wedge F2.origin = F3.origin  \wedge F3.origin = F4.origin}\;\; AllDiffFlights$

### 5. Find all pairs of employees who only work together. For example, the query should return a pair of employees 1 and 2 if employee 1 only works on flight instances on which employee 2 also works and vice versa. Return only the employee identifiers. (The employees returned must work on at least one flight instance.)




$\sigma_{C1.eID < C2.eID\; \wedge \; C1.fID = C2.fID} [(\rho_{C1} CrewAssignment) \times (\rho_{C2} CrewAssignment)]$


|C1.eid|C1.fID|C2.eid|C2.fID|
|---|---|---|---|
|X| KEP  |  Y | KEP  |
| Y  | PEP |  W | PEP  |
| Z  | FEP  | V  |  FEP |

*I don't think we can express this in relational algebra...*We would need to use something like `EXISTS` for that probably. For each CrewAssignment, I did the cross product with itself and got the ones (different crew) who worked in the same flight. I don't know if there's a way to check and enforce the **only** behavior.
Maybe:


Do another $\times$ and filter where $C3.eid = C2.eid$. We now have something like this:

|C1.eid|C1.fID|C2.eid|C2.fID|C3.eid|C3.fID|
|---|---|---|---|---|---|
|X| KEP  |  Y | KEP  |  Y | OLCO  |
| Y  | PEP |  W | PEP  |  Y | PGP  |
| Z  | FEP  | V  |  FEP |  Y | NPK  |


We'd get a good result if, for every row, we could check that the C3.fID is **IN** $\sigma_{C1.eID=eid}CrewAssignment$, and then vice versa, which I don't think it's  possible.

T2:
There's no check too. Get all the fID where eid_1 worked, let's call it A. Then, get all the fID where eid_2 worked, call it B. **IF** $A == A \cap B == B$, then eid_1 and eid_2 satisfy our result.



### 6. Find all employees (just their identifiers) who have scheduling conflicts, i.e., are scheduled to work on at least two incompatible flight instances. We consider two flight instances to be incompatible is they (1) overlap, i.e., they are scheduled on the same date and their flying time overlaps or (2) the flights happen on the same date, they do not overlap, but the destination of the earlier flight is different from the departure of the later flight, or if the destination of the earlier flight and departure of the later flight are the same, but there is less than an hour between the scheduled arrival of the earlier flight and the departure of the later flight.

eid who have scheduling conflicts:
- Scheduled to work on **at least two** incompatible flight instances:
    Incompatible:
        (1) - overlap: Scheduled on the same date and their fying time overlaps
        (2) - same date, don't overlap: but F1.destination != F2.origin OR: {
                                        F1.destination = F2.origin BUT 
                                        (F2.departureTime - F1.arriveTime) < 60
                                         }
$CrewFI := CrewAssignment \bowtie FlightInstance$

CF1.deptTime CF1.arrTime CF2.deptTime CF2.arrTime
A---B C---D --> This could happen
C---D A---B --> XOR this

Don't overlap:

Logic:
    - Bind all eID with flightInstances, then group them by two. Get all flightsInstances groups which happen on the same date, so we can compare both. But, when we do this, there will be (pseudo)duplicates in our relation. We want first to get 'GOOD' cases, which is cases of two flights that do not overlap. I'll get the good cases by checking if, for two FlightInstances F1 and F2, F1.arrivalTime < F2.departureTime, then they DON'T OVERLAP, since all flights starts and ends in the same day only. This will eliminate the (pseudo)duplicates + overlapping flights, and we then filter only the IDs of the flight + eID. This will gives us a tuple with (fA, fB, eID). Then, to get the flights that DO overlap, first we need to remove the (pseudo) duplicates from the table, filter them the same way, and subtract all good flights from all flights, which will get us the flights that DO overlap.
    
$CAFI := CrewAssignment \bowtie FlightInstance$

$SameDay := (\rho_{C1} CAFI) \bowtie_{C1.date = C2.date \; \wedge \; C1.fID \neq C2.fID \; \wedge \; C1.eID = C2.eID} (\rho_{C2} CAFI)$

$DontOverlap(fA, fB, eID) := \Pi_{C1.fID, C2.fID, C1.eID} [\sigma_{C1.arrivalTime < C2.departureTime} SameDay]$

$Cprod(iB, iA) := (\Pi_{fB} DontOverlap) \times (\Pi_{fA} DontOverlap)$

$Temp1 := \sigma_{(fA = iB\; \wedge \; fB = iA)\; \cup \; (fA = iA \; \wedge \; fB = iB)}[DontOverlap \times Cprod]$

$AllPseudodupDontOverlap(fA, fB, eID) := \Pi_{iB, iA, eID}$

$AllCases(fA, fB, eID) = \Pi_{C1.fID, c2.fID, C1.eID} \sigma_{C1.fID < C2.fID} SameDay$

$GoodFilteredCases := \sigma_{fA < fB} AllPseudodupDontOverlap$

$Violation1(eID) := \Pi_{eID}[AllCases - GoodFilteredCases]$ //At least one flight overlap

Adressing: same date, don't overlap -> fA.destination != fB.origin

(C1.fI, C1.eID, C2.eID, C2.FI, Flight.cID, Flight.flightNumber, Flight.destination, Flight.at)

$Temp1 := [\sigma_{C1.arrivalTime < C2.departureTime} SameDay] \bowtie_{C1.cID = Flight.cID \; \wedge \; C1.flightNumber = Flight.flightNumber} [\Pi_{cID, flightNumber, destination, arriveTime}Flight]$

$Temp2 := Temp1 \bowtie_{C2.cID = F2.cID \; \wedge \; C2.flightNumber = F2.flightNumber} (\rho_{F2} (\Pi_{cID, flightNumber, origin, departureTime}Flight))$

$Violation2a := \Pi_{C1.eID} [\sigma_{Flight.destination \neq F2.origin} Temp2]$ //F1.dest != F2.origin

F1.destination = F2.origin PLUS (F2.departureTime - F1.arriveTime) < 60

$Violation2b := \Pi_{C1.eID}\sigma_{F2.departureTime - Flight.arriveTime < 60}Temp2$

$Answer(eID) := Violation1 \; \cup \; Violation2a \; \cup \; Violation2b$


### 7. Find all pilots who have expertise in all the aircraft types in the database.  Return the pilot’s employee id and rank.

--> I'll consider that the keys for the table $PIlotExpertise$ are **eID, aircraftType**, together, for this question to make sense.

This could be easily done if we could just use the division operator, we need to reconstruct this operator for this query. [INFO HERE](https://cs.stackexchange.com/a/70069)

$AllAircraftTypes(aircraftType) := \Pi_{aircraftType} Aircraft$

$ExpertEmployee(eID) := \Pi_{eID} PilotExpertise -  \Pi_{eID}[((\Pi_{eID} PilotExpertise) \times AllAircraftTypes) - PilotExpertise]$

$Answer(eID, rank) := \Pi_{eID, rank} (ExpertEmployee \bowtie Employee)$


### 8. Find the airlines with the most intra-Canada flights (the origin and destination of the flight is within Canada). Return only the company codes.

I don't think it's possible to represent this using relational algebra, we would need aggregate functions. Can't think of a way to remove this by using set difference... Requires to count rows, not just values.

### 9. Return the code of the airline with the newest aircraft.

From piazza: "firstFlightDate should be used to determine the 'newest' aircraft."
So, the newest aircraft is the one with the "biggest" dates.
-> Assuming we can compare dates <-


**TODO: check if < or <=**

$NotBiggest(acID) := \Pi_{AC1.acID} \sigma_{AC1.firstFlightDate < AC2.firstFlightDate}[(\rho_{AC1} Aicraft) \times (\rho_{AC2} Aircraft)]$

$NewestAC := (\Pi_{acID} Aircraft) - NotBiggest$

$NewestACdates(acID, fdate) := \Pi_{acID, firstFlightDate}(NewestAC \bowtie Aircraft)$

$Answer(cID) := \Pi_{cID} (FlightInstance \bowtie_{date = fdate}NewestACdates)$

### 10. For every company, return the flight instances with assignment violations, i.e. the assigned ’Captain’ does not have expertise in flying the aircraft type of the plane or the distance between the departure and destination airports is greater than the range of the plane assigned to the flight instance. Return the two-character company code and the flight instance id.

The assigned ’Captain’ does not have expertise in flying the aircraft type

-> Get flightInstances where Captain does have expertise:

IF we need to use 'rank':

$FlightCap(fID, eID, role) := \Pi_{fID, CrewAssignment.eID, role}\sigma_{rank='Captain'}(CrewAssignment \bowtie Employee)$

IF we can get just with the role (I think it's the case):

$FlightCap(fID, eID) := \sigma_{role='Captain'} CrewAssignment$
              
$CaptainFlights(fID, eID, aircraftType, acID):= ((FlightCap \bowtie PilotExpertise) \bowtie (\Pi_{fID, acID}FlightInstances))$

$HasExpertise(fID, eID) := \Pi_{fID, eID}\sigma_{aircraftType = type}(CaptainFlights \bowtie (\Pi_{acID, type}Aircraft))$

$NoExpertise(fID, eID) := (\Pi_{fID, eID} CaptainFlights) - HasExpertise$

$Violations1 (cID, fID) := \Pi_{cID, fID}( NoExpertise \bowtie FlightInstance)$

SECOND violations: distance and range

$Temp1 := (\rho_{FI} FlightInstances) \bowtie_{FI.cID = F.cID\; \wedge \; FI.flightNumber = F.flightNumber} (\rho_{F} Flight)$

$FlightDetailed(fID, cID, fnum, origin, dest, acID) := \Pi_{FI.fID, FI.cID, FI.flightNumber, F.origin, F.destination, FI.acID}Temp1$

$FlightACrange(fID, cID, fnum, origin, dest, acID, range) := FlightDetailed \bowtie (\Pi_{acID, range} Aircraft)$

$Violations2a(fID, cID) := \Pi_{fID, cID}\sigma_{distance > range}[(\rho_{FC} FlightACrange) \bowtie_{FC.origin = AD.apID1 \; \wedge \; FC.dest = AD.apID2} (\rho_{AD} AirportDistance)]$

*Maybe origin-destination is not in AirportDistance, only destination-origin >:D*

$Violations2b(fID, cID) := \Pi_{fID, cID}\sigma_{distance > range}[(\rho_{FC} FlightACrange) \bowtie_{FC.origin = AD.apID2 \; \wedge \; FC.dest = AD.apID1} (\rho_{AD} AirportDistance)]$


Smash them together

$Answer(fID, cID) := Violations1 \cup Violations2a \cup Violations2b$


### 11.Find all employees (return their employee ids) who with speak French as a second language or have expertise in an aircraft with maximum speed over 700km/h at typical cruise altitude.

Seems there's a typo in the question "who with"... I consider this question is asking for all employees who speak french as a second language or (logical or) have expertise in an aircraft with maximum speed over 700
Speak french as a second language.

$FSpeakers(eID) := \Pi_{eID}\sigma_{secondary\_language='French'}Employee$

Have expertise in an arcraft with maxspeed > 700km/h

$FAircraft := \Pi_{acID, type, maxSpeed} \sigma_{maxSpeed > 700} Aircraft$

$HaveExp(eID) := \Pi_{eID} [PilotExpertise \bowtie_{PilotExpertise.aircraftType = FAircraft.type} FAircraft]$

$Answer(eID) := FSpeakers \; \cup \; HaveExp$



### 12. Return the codes of the airlines that do not have any Boeing Dreamliner (aircraft type 787-9) in their registered fleet.

Kind of difficult to get the "registered fleet" for an airline. I'm assuming every aircraft in the registered fleet has flown once, or better, have an entry in the flightInstance relation

Get all the ones who have one dreamliner

$Dreamliner(acID) := \Pi_{acID}\sigma_{manufacturer='Boeing' \wedge type='787-9'} Aircraft$

$HaveDreamliner (cID) := \Pi_{cID}[FlightInstance \bowtie Dreamliner]$

$Answer(cID) := (\Pi_{cID} Airline) - HaveDreamliner$




INTEGRITY CONSTRAINTS
==============

### 1. No flight originates and ends in the same airport.

$\sigma_{origin = destination}Flight = \emptyset$

### 2. An employee cannot be assigned multiple times to the same flight implementation.


$\sigma_{C1.fID = C2.fID \; \wedge \; C1.eID = C1.eID \; \wedge \; C1.role \neq C2.role}[(\rho_{C1} CrewAssignment) \times (\rho_{C2} CrewAssignment)] = \emptyset$

